In [1]:
import numpy as np
import pandas as pd

filename = 'imdb.csv'

def gen_df(filename):

    # Get .csv.file
    data = pd.read_csv(filename, sep=';', encoding='latin-1', escapechar='\\')

    # Read file into pandas dataframe
    df = pd.DataFrame(data)

    # Drop unwanted columns
    for i in range(44,48):
        df.drop(columns=[f'Unnamed: {i}'], inplace=True)

    df.drop(columns=['fn','wordsInTitle','url'], inplace=True)

    # Keep only observations of movie-type
    I = df['type'] == 'video.movie'
    df = df.loc[I]
    df.drop(columns=['type'], inplace=True)

    # Drop observations with missing data
    df.dropna(inplace=True)

    # Replace 0's in imdb-ratings
    df['imdbRating'] = df['imdbRating'].astype(str)
    df['imdbRating'].replace(regex=True, inplace=True,to_replace='0',value='')
    df['imdbRating'] = df['imdbRating'].astype(float)

    # Transform duration from seconds to hours
    df['duration'] = df['duration']/60**2

    # Drop years before 1920 and 2014 because of few obervations
    I = (df['year']>=1920) & (df['year']<=2013)
    df = df.loc[I]

    # Change the type of 'year' to integer
    df['year'] = df['year'].astype(int)    
    
    # Sort observations and reset index
    df.sort_values('year', inplace=True)
    df.reset_index(inplace=True)


    # Generating variable that shows the decade (as a string)
    year_list = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]

    df['decade'] = ''
    for i,start in enumerate(year_list):
        end = start+10
        df.loc[(df['year'] >= start) & (df['year'] < end), 'decade'] = f'{year_list[i]}s'

    return df

In [1]:
filename = 'imdb.csv'
df = gen_df(filename)
imdb = df.copy()

decade_list = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]

# Decade dummies
for i in decade_list:
    imdb[f'decade_{i}'] = 0
    imdb.loc[(imdb['decade'] == f'{i}s'),f'decade_{i}'] = 1

imdb['imdbRating'] = imdb['imdbRating']/10

#print(imdb.shape)

imdb = imdb.loc[(imdb['ratingCount']>=5000)]
imdb.drop(columns=['Adult','GameShow','News','RealityTV','TalkShow'], inplace=True)
#print(imdb.shape)





#23 genres



alphas = [f'alpha{i}' for i in range(1,24)]


#alphas = [globals()[alpha %] for i in range(1,24)]



#alpha = [globals()['alpha' % i] for i in range(1,24)]
print(alphas)
#print(alpha)

a = np.array([1])
b = np.array([3])
print(a@b)


def U_data(alphas,betas,delta,gamma,rho1,rho2,genres,decades,wins,duration):
    return alphas@genres+betas@decades+gamma*wins+rho1*duration+rho2*duration*duration


def R_model():
    return np.exp(U_data(alphas,betas,delta,gamma,rho1,rho2,genres,decades,wins,duration))/(1+p.exp(U_data(alphas,betas,delta,gamma,rho1,rho2,genres,decades,wins,duration)))





NameError: name 'gen_df' is not defined

In [44]:
import numpy as np
import pandas as pd
import itertools
import time
import scipy.optimize as optimize
from data_gen import gen_df 

filename = 'imdb.csv'

df = gen_df(filename)

decade_list = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]

# Decade dummies
for i in decade_list:
    df[f'decade_{i}'] = 0
    df.loc[(df['decade'] == f'{i}s'),f'decade_{i}'] = 1


#print(imdb.shape)

df = df.loc[(df['ratingCount']>=5000)]
df.drop(columns=['Adult','GameShow','News','RealityTV','TalkShow'], inplace=True)

df['duration_sqr'] = df['duration']*df['duration']

df_X = df.copy()
df_Y = pd.DataFrame(df['imdbRating'].copy())
df_Y.rename(columns = {'imdbRating':'rat_data'}, inplace=True)

#print(type(df_Y))
#print(df_Y.head())
#print(df_X.head())



drops = ['index', 'tid', 'title', 'imdbRating', 'ratingCount', 'year', 
         'nrOfGenre', 'nrOfPhotos', 'nrOfNewsArticles',  'nrOfUserReviews', 'decade']

for i in drops:
    df_X.drop(columns=[i], inplace=True)

#print(df_X.head())

df_X = df_X.reindex(['Action','Adventure','Animation','Biography','Comedy','Crime','Documentary',
                          'Drama','Family','Fantasy','FilmNoir','History','Horror','Music','Musical',
                          'Mystery','Romance','SciFi','Short','Sport','Thriller','War','Western',
                          'decade_1920','decade_1930','decade_1940','decade_1950','decade_1960',
                          'decade_1970','decade_1980','decade_1990','decade_2000','decade_2010',
                          'nrOfNominations','nrOfWins','duration','duration_sqr'], axis=1)



#print(df_X.head())
alphas = np.random.uniform(0,1,37)


df_Y['util'] = df_X@alphas
df_Y['rat_model'] = 10*np.exp(df_Y['util'])/(1+np.exp(df_Y['util']))
df_Y['sqr_diff'] = (df_Y['rat_model']-df_Y['rat_data'])**2

def sqr_diff_sum(df_X,pars):
    util = df_X@pars
    df_Y['rat_model'] = 10*np.exp(util)/(1+np.exp(util))
    df_Y['sqr_diff'] = (df_Y['rat_model']-df_Y['rat_data'])**2
    return df_Y['sqr_diff'].sum()



print(df_X.head())
print(df_Y.head())






# Parameters
alphas = np.random.uniform(0,1,37)
betas = [1,2,3]

# Score-function for testing
def u_fun(xs,pars):
        util = xs.iloc[:,0]*pars[0]+xs.iloc[:,1]*pars[1]+xs.iloc[:,29]**pars[2]
        return 10*np.exp(util)/(1+np.exp(util))

# Building dataframe containing values for score-function
df_xs = pd.DataFrame()
df_xs = df.iloc[:,13:41]
df_xs['Duration'] = df['duration']
df_xs['Rating'] = df['imdbRating']

# Function that calculates minimum squared distance
def obj_fun(df_xs,pars):
        df_xs['Est_rating'] = u_fun(df_xs,pars)
        df_xs['Difference'] = (df_xs['Rating']-df_xs['Est_rating'])**2
        print(df_xs[['Est_rating','Rating','Difference']].head())
        return df_xs['Difference'].sum()

# Testing from here
#obj_fun(df_xs,alphas)
#obj_fun(df_xs,[0,0,0])



    Action  Adventure  Animation  Biography  Comedy  Crime  Documentary  \
3      0.0        0.0        0.0        0.0     0.0    0.0          0.0   
13     0.0        0.0        0.0        0.0     1.0    0.0          0.0   
19     0.0        0.0        0.0        0.0     0.0    0.0          0.0   
21     0.0        0.0        0.0        0.0     0.0    0.0          1.0   
32     0.0        0.0        0.0        0.0     1.0    0.0          0.0   

    Drama  Family  Fantasy  ...  decade_1960  decade_1970  decade_1980  \
3     0.0     0.0      0.0  ...            0            0            0   
13    1.0     1.0      0.0  ...            0            0            0   
19    0.0     0.0      0.0  ...            0            0            0   
21    0.0     0.0      0.0  ...            0            0            0   
32    0.0     0.0      0.0  ...            0            0            0   

    decade_1990  decade_2000  decade_2010  nrOfNominations  nrOfWins  \
3             0            0    

In [ ]:
"""
min_fun = lambda a: obj_fun(df_xs,a)

bnds = ((0,1),(0,1),(0,1))

result = optimize.minimize(min_fun,alphas,method='SLSQP',bounds=bnds)

print(alphas,result.x)

print(obj_fun(df_xs,alphas),obj_fun(df_xs,result.x))

"""

"""
n = 300

#A1 = np.random.uniform(1,20,size=n)
#A2 = np.linspace(0,10,n)
#A3 = np.linspace(0,10,n)
start = time.time()

A1_grid, A2_grid, A3_grid = np.meshgrid(A1,A2,A3,indexing='ij')
A=A1_grid+A2_grid+A3_grid
print(np.max(A))

end = time.time()-start

print(f'{end:3.f} seconds')

iterator = itertools.combinations(A1, 3)

high = 0
time_list = list()

for k in range(10):

        start = time.time()

        iterator = itertools.combinations(enumerate(A1), 3)
        for i in iterator:
            print(i,i[0:],i[1:])
            
            test1 = i[1][0]/i[1][1]+i[1][2]
            test2 = i[0]*i[1]*i[2]
            
        if test2 < 20:
                if test1 > high:
                        high = test1
                        high_i = i
                        
        if j%100000 == 0:
                part = time.time()-start
                print(f'Iteratrion: {j} after {part:.2f} seconds')
        
        end = time.time()-start
        time_list.append(end)

avg = sum(time_list)/len(time_list)
print(f'{avg:.3f} seconds')

#print(f'{end:.3f} seconds, highest value is {high:.3f} with i: {high_i}')

for k in range(1,4):
    start = time.time()

    list_1 = np.random.uniform(1,20,size=n)
    list_2 = np.random.uniform(1,5,size=n)

    iterator = itertools.combinations(enumerate(list_2), 3)

    hi = 0
    indeces = list()

    for i in iterator:
        duration = sum([j[1] for j in i])        
        if duration >= 12 and duration <= 14:
            score = sum([list_1[j[0]] for j in i])
            if score > hi:
                hi = score
                dur = duration
                indeces = [j[0] for j in i]

    end = time.time() - start

    text = f'Highest value is {hi:.2f} \n'
    text += f'The duration is {dur:.2f} \n'
    text += f'This took {end:.1f} seconds'

    print(f'Iteration: {k}')
    print(text)
"""

